In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`io.delta:delta-spark_2.13:3.1.0`

import $ivy.$
import $ivy.$

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()

import spark.implicits._

val tablePath = "../local/delta-scala"


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/30 18:12:17 WARN Utils: Your hostname, DESKTOP-CN01VS3 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/30 18:12:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/30 18:12:17 INFO SparkContext: Running Spark version 3.5.0
25/03/30 18:12:17 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/03/30 18:12:17 INFO SparkContext: Java version 17.0.14
25/03/30 18:12:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/30 18:12:17 INFO ResourceUtils: ==============================================================
25/03/30 18:12:17 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/30 18:12:17 INFO ResourceUtils: ==============================================================
25/03/30 18:12:17 INFO SparkCon

import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@5ccc64ea
import spark.implicits._
tablePath: String = "../local/delta-scala"

In [3]:
val df = Seq(
  (8, "bat"),
  (64, "mouse"),
  (1, "horse")
).toDF("number", "word")

25/03/30 18:12:23 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/30 18:12:23 INFO SharedState: Warehouse path is 'file:/home/eyal/code/notebooks/spark-warehouse'.
25/03/30 18:12:24 INFO CodeGenerator: Code generated in 169.255807 ms


df: org.apache.spark.sql.package.DataFrame = [number: int, word: string]

In [4]:
df.write
  .format("delta")
  .mode("overwrite")  // or append
  .save(tablePath)

25/03/30 18:12:26 INFO DelegatingLogStore: LogStore `LogStoreAdapter(io.delta.storage.HDFSLogStore)` is used for scheme `file`
25/03/30 18:12:26 INFO DeltaLog: Loading version 0.
25/03/30 18:12:27 INFO DeltaLogFileIndex: Created DeltaLogFileIndex(JSON, numFilesInSegment: 1, totalFileSize: 1793)
25/03/30 18:12:27 INFO DataSourceStrategy: Pruning directories with: 
25/03/30 18:12:27 INFO FileSourceStrategy: Pushed Filters: 
25/03/30 18:12:27 INFO FileSourceStrategy: Post-Scan Filters: (isnotnull(protocol#26.minReaderVersion) OR isnotnull(metaData#25.id))
25/03/30 18:12:27 INFO CodeGenerator: Code generated in 70.94175 ms
25/03/30 18:12:27 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 376.0 B, free 2.2 GiB)
25/03/30 18:12:27 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 2.2 GiB)
25/03/30 18:12:27 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 10.255.255.254:40375 (size: 34.4 KiB, free: 2.2

In [6]:
spark.read.format("delta").load(tablePath).show()

25/03/30 18:12:45 INFO PrepareDeltaScan: DELTA: Filtering files for query
25/03/30 18:12:45 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
25/03/30 18:12:45 INFO DAGScheduler: Got job 13 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 50 output partitions
25/03/30 18:12:45 INFO DAGScheduler: Final stage: ResultStage 22 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
25/03/30 18:12:45 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 21)
25/03/30 18:12:45 INFO DAGScheduler: Missing parents: List()
25/03/30 18:12:45 INFO DAGScheduler: Submitting ResultStage 22 (MapPartitionsRDD[57] at $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128), which has no missing parents
25/03/30 18:12:45 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 709.1 KiB, free 2.2 GiB)
25/03/30 18:12:45 INFO MemoryStore: Block broadcast_20_piec

+------+-----+
|number| word|
+------+-----+
|     1|horse|
|    64|mouse|
|     8|  bat|
+------+-----+

